In [18]:
import os
import json
import psycopg2
from psycopg2.extras import execute_values, Json
from dotenv import load_dotenv

In [19]:
# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------
# Update with your actual Supabase connection string
# Settings -> Database -> Connection String -> URI
load_dotenv()
DB_CONNECTION_STRING = os.getenv('SUPABASE_DB_CS') 
JSON_FILE_PATH = "data/tcas_courses.json" 
TABLE_NAME = "tcas69_programs"
BATCH_SIZE = 500

# ---------------------------------------------------------
# 1. DATABASE SETUP
# ---------------------------------------------------------
def setup_database(cursor):
    print("🔧 Setting up database schema...")

    # 1. Enable pg_trgm extension
    # This is CRITICAL for fast 'LIKE %query%' operations
    cursor.execute("CREATE EXTENSION IF NOT EXISTS pg_trgm;")

    # 2. Create the table with extracted columns + full JSON backup
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
            id BIGINT PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
            program_id TEXT,
            
            -- Thai & English Names (Extracted for Search)
            university_name_th TEXT,
            university_name_en TEXT,
            faculty_name_th TEXT,
            faculty_name_en TEXT,
            field_name_th TEXT,
            field_name_en TEXT,
            program_name_th TEXT,
            program_name_en TEXT,
            major_name_th TEXT,
            major_name_en TEXT,
            
            -- Metrics
            seats INT, -- Mapped from number_acceptance_mko2
            cost TEXT,
            graduate_rate TEXT,
            
            -- Full Data Backup
            full_details JSONB,
            
            updated_at TIMESTAMPTZ DEFAULT NOW()
        );
    """)

    # 3. Create Trigram Indexes
    # These specific indexes make 'LIKE %...%' queries 100x faster
    print("⚡ Creating optimized indexes for partial text search...")
    
    searchable_columns = [
        "university_name_th", "university_name_en",
        "faculty_name_th", "faculty_name_en",
        "field_name_th", "field_name_en",
        "program_name_th", "program_name_en",
        "major_name_th", "major_name_en"
    ]
    
    for col in searchable_columns:
        # The name of the index
        index_name = f"idx_{TABLE_NAME}_{col}_trgm"
        # gin_trgm_ops is the magic operator for LIKE search
        cursor.execute(f"""
            CREATE INDEX IF NOT EXISTS {index_name} 
            ON {TABLE_NAME} USING gin ({col} gin_trgm_ops);
        """)

# ---------------------------------------------------------
# 2. MIGRATION LOGIC
# ---------------------------------------------------------
def migrate_data():
    conn = None
    try:
        # Connect to Supabase
        conn = psycopg2.connect(DB_CONNECTION_STRING)
        cur = conn.cursor()
        
        # Run Schema Setup
        setup_database(cur)
        conn.commit()

        # Load JSON File
        print(f"📂 Reading {JSON_FILE_PATH}...")
        try:
            with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
                data = json.load(f)
        except FileNotFoundError:
            print(f"❌ Error: File '{JSON_FILE_PATH}' not found.")
            return

        print(f"🚀 Found {len(data)} records. Starting migration...")

        # Prepare SQL Query
        insert_query = f"""
            INSERT INTO {TABLE_NAME} (
                program_id, 
                university_name_th, university_name_en, 
                faculty_name_th, faculty_name_en, 
                field_name_th, field_name_en,
                program_name_th, program_name_en,
                major_name_th, major_name_en,
                seats, cost, graduate_rate, 
                full_details
            )
            VALUES %s
        """

        batch = []
        total_inserted = 0

        for item in data:
            # Safely extract data with defaults using .get()
            record = (
                item.get('program_id'),
                item.get('university_name_th'),
                item.get('university_name_en'),
                item.get('faculty_name_th'),
                item.get('faculty_name_en'),
                item.get('field_name_th'),
                item.get('field_name_en'),
                item.get('program_name_th'),
                item.get('program_name_en'),
                item.get('major_name_th', ''),
                item.get('major_name_en', ''),
                item.get('number_acceptance_mko2', 0), # Mapping specific field
                item.get('cost', ''),
                item.get('graduate_rate', ''),
                Json(item) # Store the entire original object as JSONB
            )
            batch.append(record)

            # Execute in batches (better for network/performance)
            if len(batch) >= BATCH_SIZE:
                execute_values(cur, insert_query, batch)
                total_inserted += len(batch)
                print(f"   Processed {total_inserted} rows...")
                batch = []

        # Insert remaining rows
        if batch:
            execute_values(cur, insert_query, batch)
            total_inserted += len(batch)

        conn.commit()
        print(f"✅ Success! Migrated {total_inserted} records.")
        print("---------------------------------------------------")
        print("💡 Example SQL query for your app:")
        print(f"SELECT program_name_th, cost FROM {TABLE_NAME} WHERE faculty_name_th LIKE '%วิศวะ%';")

    except Exception as e:
        print("❌ Critical Error:", e)
        if conn:
            conn.rollback()
    finally:
        if conn:
            cur.close()
            conn.close()

In [ ]:
migrate_data()

🔧 Setting up database schema...
⚡ Creating optimized indexes for partial text search...
📂 Reading data/tcas_courses.json...
🚀 Found 4647 records. Starting migration...
   Processed 500 rows...
   Processed 1000 rows...
   Processed 1500 rows...
   Processed 2000 rows...
   Processed 2500 rows...
   Processed 3000 rows...
   Processed 3500 rows...
   Processed 4000 rows...
   Processed 4500 rows...
✅ Success! Migrated 4647 records.
---------------------------------------------------
💡 Example SQL query for your app:
SELECT program_name_th, cost FROM tcas69_programs WHERE faculty_name_th LIKE '%วิศวะ%';
